In [2]:
from mesa.batchrunner import BatchRunner, BatchRunnerMP
from model import *
import pickle
import numpy as np
import uuid

In [3]:
from customizeTime import *
import env_settings

'''
class EnvIncreaseAgent(GenAgent):
    def __init__(self, unique_id, model, generation_num, gen_type = None, gen_info = None):
        super().__init__(unique_id, model, generation_num, gen_type, gen_info)
        self.schedule = RandomActivationMutationNoConstrain(self)
'''       
    
class EnvIncreaseModel(GenModel):
    def init_env(self):
        return env_settings.increase_env
    def get_schedule(self):
        return RandomActivationMutationNoConstrain
    #def get_agent_config(self):
    #    return EnvIncreaseAgent
    
    def model_level_record_init(self):
        data_file_name = "./EnvIncrease" + str(uuid.uuid1())
        print('data in file ', data_file_name)
        self.data_file = open(data_file_name, 'w')
        self.data_file.write('#' + '\t'.join(('env', 'min', 'mean', 'max', 'gen_type', 'agent num')) + '\n')
        
    def collect_data(self, gen_type):
        env = self.cur_press
        gene = [agent.gen_info for agent in self.schedule.agents if agent.gen_type == gen_type]
        min_v = np.min([min(g) for g in gene])
        max_v = np.max([max(g) for g in gene])
        mean_v = np.mean([np.mean(g) for g in gene])
        print((env, min_v, mean_v, max_v, gen_type, len(gene)))
        return (env, min_v, mean_v, max_v, gen_type, len(gene))
        
        
    def step(self):
        super().step()
        data = self.collect_data(0)
        self.data_file.write('\t'.join([str(d) for d in data]) + '\n')
        data = self.collect_data(1)
        self.data_file.write('\t'.join([str(d) for d in data]) + '\n')

In [3]:
model = EnvIncreaseModel(2000, 100, 100)
for i in range(10):
    model.step()

data in file  ./EnvIncreasefb1ee608-9b6b-11e9-bed9-f2189850a77c
0 population size:  2000
(0.5, -0.1920962170374727, 0.5004840266780376, 1.206827316647729, 0, 1757)
(0.5, -0.2563854928736628, 0.5003018376252143, 1.1894658632475892, 1, 1694)
1 population size:  3451
(0.505, -0.2801689027081159, 0.5009501525907593, 1.206827316647729, 0, 2757)
(0.505, -0.2563854928736628, 0.500445020993347, 1.2848947582072825, 1, 2628)
2 population size:  5385
(0.51, -0.2801689027081159, 0.5010813137424671, 1.2889532287481658, 0, 3754)
(0.51, -0.2858103639944348, 0.5007711443003054, 1.3048380655903717, 1, 3524)
3 population size:  7278
(0.515, -0.2801689027081159, 0.5015829332726631, 1.2889532287481658, 0, 4428)
(0.515, -0.3203961891472137, 0.5005698332302219, 1.3048380655903717, 1, 4249)
4 population size:  8677
(0.52, -0.2801689027081159, 0.5021656394953672, 1.2889532287481658, 0, 4139)
(0.52, -0.2748406769448653, 0.5005900128091281, 1.3048380655903717, 1, 3935)
5 population size:  8074
(0.525, -0.280168

In [4]:
def compute_ratio(model):
    agent_types = [agent.gen_type for agent in model.schedule.agents]
    if len(agent_types): return 1 - (sum(agent_types) / len(agent_types))
    else:return 0
fixed_params = {'N':2000, 'width':100, 'height':100}

variable_params = {'init_ratio': [0.5]}

batch_run = BatchRunnerMP(
    EnvIncreaseModel, 5,
    variable_parameters=variable_params,
    fixed_parameters=fixed_params,
    iterations=10,
    max_steps=200,
    model_reporters={"two type ratio": compute_ratio}
)
batch_run.run_all()

0it [00:00, ?it/s]

data in file  ./EnvIncrease8571cc80-9b6c-11e9-99de-f2189850a77c
data in file  ./EnvIncrease85723a1c-9b6c-11e9-8085-f2189850a77c
data in file  ./EnvIncrease8572c39c-9b6c-11e9-bd9b-f2189850a77c
data in file  ./EnvIncrease857322ec-9b6c-11e9-842d-f2189850a77c
data in file  ./EnvIncrease8573ce36-9b6c-11e9-8a80-f2189850a77c
0 population size:  2000
0 population size:  2000
0 population size:  2000
0 population size:  2000
0 population size:  2000
(0.5, -0.267542008517457, 0.5006616125343462, 1.1719049693437191, 0, 1795)
(0.5, -0.22407148853869607, 0.5007810433906049, 1.177421849437334, 1, 1657)
1 population size:  3452
(0.5, -0.22402268873107456, 0.5016334628172058, 1.2928296167262454, 0, 1795)
(0.5, -0.15732501382861241, 0.49929133924942365, 1.1972133370474927, 0, 1810)
(0.5, -0.32603399131532473, 0.5006091709239213, 1.1600737306789832, 0, 1782)
(0.5, -0.18826564087178432, 0.5009814055211945, 1.1896881460362212, 1, 1673)
1 population size:  3468
(0.5, -0.2010288777268764, 0.4997209227511716

(0.54, -0.3006376192382442, 0.4977946627257901, 1.2553981413707245, 0, 4057)
(0.54, -0.2205020058934702, 0.5004624895216875, 1.251499568536176, 1, 3888)
9 population size:  7945
(0.54, -0.23787486848515282, 0.504355294776357, 1.3939186441237938, 0, 4030)
(0.54, -0.2681915911109325, 0.4985767101319968, 1.345955474463509, 1, 3965)
9 population size:  7995
(0.545, -0.31265206542250357, 0.5032703148419436, 1.2899894088521515, 0, 4130)
(0.545, -0.3132474707549021, 0.4985765398741992, 1.2892945457617853, 1, 3916)
10 population size:  8046
(0.545, -0.30008598071166587, 0.5073704416973321, 1.2944588761255043, 0, 4160)
(0.545, -0.33135047966704523, 0.5032563917496252, 1.3716643847026715, 1, 3912)
10 population size:  8072
(0.545, -0.3824672692096033, 0.5024879866740465, 1.2623720717557307, 0, 4017)
(0.545, -0.330570472428066, 0.49965752663512486, 1.4101813166641308, 1, 3999)
10 population size:  8016
(0.545, -0.3006376192382442, 0.49906830914971123, 1.2553981413707245, 0, 4100)
(0.545, -0.22050

18 population size:  7925
(0.585, -0.3006376192382442, 0.5106750950414626, 1.3139312326401726, 0, 3927)
(0.585, -0.40761829138302424, 0.5032902903366524, 1.2948600787581035, 1, 4035)
18 population size:  7962
(0.585, -0.32058568515775965, 0.5133097296236251, 1.4256032638900944, 0, 3899)
(0.585, -0.4333082350912264, 0.5005346678023797, 1.337122561550697, 1, 4077)
18 population size:  7976
(0.59, -0.33303887992328096, 0.5159737926679524, 1.2773084752584354, 0, 4004)
(0.59, -0.4202639136854401, 0.4977152901786819, 1.2589555383607527, 1, 3913)
19 population size:  7917
(0.59, -0.41966802498736283, 0.5102385153534162, 1.4355457445611057, 0, 4085)
(0.59, -0.3160465811521272, 0.5193089020570529, 1.3574700409463272, 0, 3909)
(0.59, -0.330570472428066, 0.5018409393773042, 1.3438948367759393, 1, 3829)
19 population size:  7914
(0.59, -0.39566852207885256, 0.5034599590968877, 1.3716643847026715, 1, 3999)
19 population size:  7908
(0.59, -0.3006376192382442, 0.5126583734349763, 1.3139312326401726,

(0.63, -0.4093098953937633, 0.5048031371815996, 1.3098203258431704, 1, 4000)
27 population size:  7868
(0.63, -0.31354266223736255, 0.5270614352896044, 1.3191336319484168, 0, 3729)
(0.63, -0.4907966987232766, 0.50331923279906, 1.337122561550697, 1, 4201)
27 population size:  7930
(0.635, -0.39746268977015414, 0.5315437031867942, 1.3625635722193021, 0, 4017)
(0.635, -0.39600386846634084, 0.5009669650578625, 1.3082332139472022, 1, 3829)
28 population size:  7846
(0.635, -0.34083757764476613, 0.525415279294639, 1.3392139867340236, 0, 3982)
(0.635, -0.4694917666859001, 0.5045916103768617, 1.3438948367759393, 1, 3862)
28 population size:  7844
(0.635, -0.3674009217556281, 0.5350159073101767, 1.4012162333949805, 0, 3942)
(0.635, -0.5062369068959892, 0.505718005789375, 1.3716643847026715, 1, 3951)
28 population size:  7893
(0.635, -0.3140927017832612, 0.5314930653708678, 1.2829137630420755, 0, 3876)
(0.635, -0.4093098953937633, 0.5051474808877108, 1.3098203258431704, 1, 4037)
28 population si

...

(0.865, -0.38696576528410065, 0.541024073946459, 1.367743298500275, 1, 1535)
74 population size:  7659
(0.915, -0.6008185174737887, 0.7614400078487218, 1.5664942817387375, 0, 7022)
(0.915, -0.3651751660149348, 0.5443865519649537, 1.3578145809718771, 1, 756)
84 population size:  7778
(0.9, -0.41362932574245176, 0.7410601262825558, 1.5060976577038954, 0, 6614)
(0.9, -0.468775245685754, 0.5575954528075621, 1.3253066326409801, 1, 1075)
81 population size:  7689
(0.87, -0.49210656735628205, 0.7175794714727263, 1.6443185620598058, 0, 5457)
(0.87, -0.45033575781212454, 0.5281073329561865, 1.5298232705195456, 1, 2053)
75 population size:  7510
(0.9, -0.360457102807043, 0.7466670744887254, 1.4619448291088377, 0, 6698)
(0.9, -0.36742267911888926, 0.5543223228387073, 1.4337019256209922, 1, 922)
81 population size:  7620
(0.9199999999999999, -0.6008185174737887, 0.7654418951086575, 1.5664942817387375, 0, 7047)
(0.87, -0.32496455344829583, 0.7138463289687996, 1.3973303787163482, 0, 6253)
(0.9199999

89 population size:  7723
(0.96, -0.3482243335367591, 0.7946343564465925, 1.622325379386529, 0, 7511)
(0.96, -0.3858729350665307, 0.5539142477708008, 1.2870922399230296, 1, 211)
93 population size:  7722
(0.91, -0.30695979574719773, 0.7500944155908729, 1.5158846942536865, 0, 6918)
(0.91, -0.4063124282109038, 0.5532990886977713, 1.3680569609974984, 1, 779)
83 population size:  7697
(0.9450000000000001, -0.34544240275640636, 0.7839791537946816, 1.4996916054310565, 0, 7198)
(0.9450000000000001, -0.47940586647133043, 0.5704034061333549, 1.34562879289579, 1, 469)
90 population size:  7667
(0.915, -0.28868410813397055, 0.752970639374491, 1.4944521139375693, 0, 6636)
(0.915, -0.4021521971475045, 0.5365192360362374, 1.452298580539511, 1, 926)
84 population size:  7562
(0.9450000000000001, -0.31236614746891006, 0.7826602796207098, 1.5123543267924677, 0, 7345)
(0.9450000000000001, -0.2867091585158843, 0.564473756020035, 1.367308243070325, 1, 373)
90 population size:  7718
(0.9650000000000001, -0

(0.9550000000000001, -0.4021521971475045, 0.5407286579677395, 1.411133792563016, 1, 392)
92 population size:  7719
(0.985, -0.28524561943755156, 0.8115106121144814, 1.535963256322292, 0, 7580)
(0.985, -0.4233151765191094, 0.5812313922403632, 1.2271957132867388, 1, 125)
98 population size:  7705
(1.005, -0.24066811320340903, 0.8261108462119796, 1.622325379386529, 0, 7615)
(1.005, -0.1170823588230118, 0.5890315858438502, 1.2315960678291675, 1, 55)
102 population size:  7670
(0.9550000000000001, -0.33996406467612583, 0.7881524988218132, 1.6077319771730874, 0, 7343)
(0.9550000000000001, -0.3310457620749197, 0.5655148074501, 1.3550516862331219, 1, 341)
92 population size:  7684
(0.99, -0.3501298110795078, 0.8243489121993555, 1.4996916054310565, 0, 7574)
(0.99, -0.29163135932685114, 0.591243212201724, 1.3248173694140906, 1, 171)
99 population size:  7745
(0.96, -0.239314553727062, 0.7859383327635058, 1.4996813410083587, 0, 7330)
(0.96, -0.4021521971475045, 0.5438676693352231, 1.4111337925630

107 population size:  7751
(1.0, -0.3240973045743851, 0.8198381045376327, 1.6017124024551892, 0, 7574)
(1.0, -0.4021521971475045, 0.5699892254903627, 1.3114383961698732, 1, 95)
101 population size:  7669
(1.05, -0.25753824105669976, 0.8592621826338052, 1.6589138244368236, 0, 7640)
(1.05, -0.0214032280079596, 0.605860935764979, 1.1899306226246138, 1, 12)
111 population size:  7652
(1.03, -0.1934132550183675, 0.8464491953625367, 1.647618112638378, 0, 7729)
(1.03, -0.24092033575681432, 0.6071573642770105, 1.2048308586877488, 1, 20)
107 population size:  7749
(1.0, -0.3028644264441558, 0.8191477894682169, 1.5457408042445726, 0, 7521)
(1.0, -0.19583816086438924, 0.5719036705611431, 1.3195422538148278, 1, 95)
101 population size:  7616
(1.0350000000000001, -0.3501298110795078, 0.8595388963042647, 1.66422382136055, 0, 7617)
(1.0350000000000001, -0.15957629931140913, 0.6072496811473953, 1.3388868887638667, 1, 46)
108 population size:  7663
(1.005, -0.3240973045743851, 0.8232724414460099, 1.495

(1.0750000000000002, -0.0003730284254224586, 0.6072488852015514, 1.2314203554798384, 1, 2)
116 population size:  7728
(1.045, -0.3485760713689106, 0.8525283077406237, 1.5546607316420795, 0, 7732)
(1.045, -0.05851800709955511, 0.5809395053161416, 1.2657485603304788, 1, 14)
110 population size:  7746
(1.0750000000000002, -0.20782803215309512, 0.8790395611805941, 1.5682529503075566, 0, 7641)
(1.0750000000000002, 0.08746302260413363, 0.6589417615971067, 1.0953373745692676, 1, 3)
116 population size:  7644
(1.045, -0.24086875102260324, 0.8479838136131153, 1.5783986290801297, 0, 7612)
(1.045, -0.0856734434701557, 0.5819356450940198, 1.2125660164859513, 1, 35)
110 population size:  7647
(1.08, -0.17605648833770007, 0.8883047845013339, 1.6573481564126291, 0, 7753)
(1.08, -0.0003730284254224586, 0.5764595657370452, 1.2314203554798384, 1, 3)
117 population size:  7756
(1.05, -0.3485760713689106, 0.8552820000623859, 1.5546607316420795, 0, 7703)
(1.05, -0.05851800709955511, 0.585248043732945, 1.26